In [31]:
import os
from tqdm import tqdm
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import average_precision_score, precision_recall_curve, classification_report
from xgboost import XGBClassifier
from collections import Counter
import tensorflow as tf

In [32]:
# --- Setup
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
OUT_DIR = "/content/flight_optuna_best"
os.makedirs(OUT_DIR, exist_ok=True)

# --- Synthetic balanced dataset
N_SEQ, TIMESTEPS, FEATURES = 8000, 60, 7
def generate_sequence(is_incident, timesteps=60):
    t = np.linspace(0,1,timesteps)
    alt = 10000 + 200*np.sin(2*np.pi*t) + np.random.normal(0,8,timesteps)
    spd = 250 + 8*np.cos(2*np.pi*t) + np.random.normal(0,3,timesteps)
    pitch, roll, yaw = np.random.normal(0,0.8,timesteps), np.random.normal(0,0.8,timesteps), np.random.normal(0,0.5,timesteps)
    eng, thrust = 200+np.random.normal(0,2,timesteps), 70+np.random.normal(0,2,timesteps)
    seq = np.vstack([alt,spd,pitch,roll,yaw,eng,thrust]).T
    weather, phase, maint = random.choice(["clear","rain","storm","fog","turbulence"]), random.choice(["cruise","takeoff","landing","climb","descent"]), np.random.randint(0,3)
    if is_incident:
        pat = random.choice(["rapid_descent","stall","engine_fail"])
        if pat=="rapid_descent": seq[:,0]-=np.linspace(0,np.random.randint(1000,1500),timesteps)
        if pat=="stall": seq[:,1]-=np.linspace(0,np.random.randint(60,120),timesteps); seq[:,2]+=np.linspace(0,np.random.randint(4,8),timesteps)
        if pat=="engine_fail": seq[:,5]+=np.linspace(0,np.random.randint(100,150),timesteps); seq[:,6]-=np.linspace(0,np.random.randint(30,50),timesteps)
        maint+=np.random.randint(1,2); weather=random.choice(["storm","turbulence","fog"])
    return seq, weather, phase, maint, int(is_incident)

seqs, metas, labels = [], [], []
for i in tqdm(range(N_SEQ)):
    seq,w,p,m,y = generate_sequence(i>=N_SEQ//2)
    seqs.append(seq); metas.append((w,p,m)); labels.append(y)
seqs=np.array(seqs); metas=pd.DataFrame(metas,columns=["weather","phase","maint"]); labels=np.array(labels)

100%|██████████| 8000/8000 [00:00<00:00, 16345.86it/s]


In [33]:
N, T, F = seqs.shape

In [34]:
def slope(y):
    # simple least-squares slope vs time index
    x = np.arange(len(y))
    x_mean = x.mean()
    y_mean = y.mean()
    denom = ((x - x_mean) ** 2).sum() + 1e-9
    return float(((x - x_mean) * (y - y_mean)).sum() / denom)

In [35]:
def exceedance(y, thr, greater=True):
    if greater:
        return float((y > thr).mean())
    else:
        return float((y < thr).mean())

In [36]:
def rolling_std(y, w=5):
    if len(y) < w:
        return float(np.std(y))
    # naive rolling std average
    rs = []
    for i in range(len(y)-w+1):
        rs.append(np.std(y[i:i+w]))
    return float(np.mean(rs))

In [37]:
def ts_features(seq):
    # seq: (T, 7)
    alt, spd, pitch, roll, yaw, eng, thrust = [seq[:, i] for i in range(7)]
    feats = {}
    # Generic per-channel stats
    def add_stats(prefix, y):
        feats[f"{prefix}_mean"] = float(np.mean(y))
        feats[f"{prefix}_std"] = float(np.std(y))
        feats[f"{prefix}_min"] = float(np.min(y))
        feats[f"{prefix}_max"] = float(np.max(y))
        feats[f"{prefix}_last"] = float(y[-1])
        feats[f"{prefix}_p10"] = float(np.percentile(y,10))
        feats[f"{prefix}_p90"] = float(np.percentile(y,90))
        feats[f"{prefix}_slope"] = slope(y)
        feats[f"{prefix}_rollstd5"] = rolling_std(y, w=5)
        diffs = np.diff(y)
        feats[f"{prefix}_dmean"] = float(np.mean(diffs))
        feats[f"{prefix}_dstd"] = float(np.std(diffs))
        feats[f"{prefix}_dmax"] = float(np.max(diffs))
        feats[f"{prefix}_dmin"] = float(np.min(diffs))
    add_stats("alt", alt)
    add_stats("spd", spd)
    add_stats("pitch", pitch)
    add_stats("roll", roll)
    add_stats("yaw", yaw)
    add_stats("eng", eng)
    add_stats("thrust", thrust)

    # Domain-inspired exceedances and combos
    feats["alt_drop_>8mps_frac"] = exceedance(np.diff(alt), -8, greater=False)
    feats["spd_loss_>2mps_frac"] = exceedance(np.diff(spd), -2, greater=False)
    feats["pitch_up_>3deg_frac"] = exceedance(pitch, 3, greater=True)
    feats["roll_abs_mean"] = float(np.mean(np.abs(roll)))
    feats["yaw_rate_std"] = float(np.std(np.diff(yaw)))
    feats["eng_increase_frac"] = exceedance(np.diff(eng), 1.5, greater=True)
    feats["thrust_drop_frac"] = exceedance(np.diff(thrust), -1.5, greater=False)
    feats["energy_proxy"] = float(np.mean(spd) + 0.003*np.mean(alt))
    feats["energy_drop_frac"] = exceedance(np.diff(spd + 0.003*alt), -0.8, greater=False)

    # Patterns that match your incident injections
    feats["rapid_descent_score"] = float((alt[0] - alt[-1]) / max(1, T))
    feats["stall_like_score"] = float((spd[0] - spd[-1]) / max(1, T) + np.maximum(pitch,0).mean())
    feats["engine_fail_like"] = float((eng[-1] - eng[0]) - (thrust[-1] - thrust[0]))

    return feats

In [38]:
# Build feature table
rows = []
for i in range(N):
    feats = ts_features(seqs[i])
    rows.append(feats)
X_seq = pd.DataFrame(rows)

In [39]:
# Merge with metadata
X = pd.concat([X_seq.reset_index(drop=True), metas.reset_index(drop=True)], axis=1)
y = labels.astype(int)

In [40]:
# Train/valid split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

In [41]:
# ColumnTransformer: one-hot for categorical meta, passthrough numeric
cat_cols = ["weather", "phase"]
num_cols = [c for c in X.columns if c not in cat_cols]

In [42]:
pre = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

In [43]:
# Class weighting for balanced/near-balanced; if exactly 50/50 it ~1
ctr = Counter(y_train)
scale_pos_weight = ctr[0] / max(1, ctr[1])
print("scale_pos_weight:", scale_pos_weight)

scale_pos_weight: 1.0


In [44]:
clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",
    tree_method="hist",
    learning_rate=0.05,
    n_estimators=3000,        # high cap; use early stopping
    max_depth=6,
    min_child_weight=2,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.2,
    reg_alpha=0.0,
    scale_pos_weight=scale_pos_weight,
    n_jobs=0,
    random_state=SEED
)

In [45]:
pipe = Pipeline([
    ("pre", pre),
    ("xgb", clf),
])

In [46]:
# Fit with early stopping (needs eval_set after preprocessing)
Xtr_mat = pre.fit_transform(X_train, y_train)
Xva_mat = pre.transform(X_valid)

In [48]:
pipe.named_steps["xgb"].fit(
    Xtr_mat, y_train,
    eval_set=[(Xtr_mat, y_train), (Xva_mat, y_valid)],
    verbose=100
)

[0]	validation_0-aucpr:0.99865	validation_1-aucpr:0.99794
[100]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[200]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[300]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[400]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[500]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[600]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[700]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[800]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[900]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[1000]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[1100]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[1200]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[1300]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[1400]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[1500]	validation_0-aucpr:1.00000	validation_1-aucpr:1.00000
[1600]	validation_0-aucpr:1.00000	va

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'aucpr'


In [49]:
# Evaluate
proba = pipe.named_steps["xgb"].predict_proba(Xva_mat)[:, 1]
ap = average_precision_score(y_valid, proba)
print(f"AUC-PR: {ap:.6f}")

AUC-PR: 1.000000


In [50]:
# Threshold selection with guardrails (avoid degenerate all-positive)
precision, recall, thresholds = precision_recall_curve(y_valid, proba)
f1s = 2 * precision * recall / (precision + recall + 1e-12)

In [52]:
def pos_rate_at_threshold(t):
    return float((proba >= t).mean())

In [53]:
candidates = []
for i, t in enumerate(np.r_[thresholds, thresholds[-1] if len(thresholds) else 0.5]):
    prate = pos_rate_at_threshold(t)
    if 0.15 <= prate <= 0.6:  # adjust to your intended alert rate
        candidates.append((f1s[i], t))
if candidates:
    thr = max(candidates, key=lambda x: x[0])[1]
else:
    # fallback: closest to target positive rate
    target_pr = 0.3
    thr = sorted(thresholds, key=lambda t: abs(pos_rate_at_threshold(t)-target_pr))[0] if len(thresholds) else 0.5

In [54]:
print(f"Chosen threshold: {thr:.4f}, positive rate={pos_rate_at_threshold(thr):.3f}")
pred = (proba >= thr).astype(int)
print(classification_report(y_valid, pred, digits=4))

Chosen threshold: 0.8473, positive rate=0.500
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       800
           1     1.0000    1.0000    1.0000       800

    accuracy                         1.0000      1600
   macro avg     1.0000    1.0000    1.0000      1600
weighted avg     1.0000    1.0000    1.0000      1600



In [57]:
import joblib, os
joblib.dump(pre, "preprocessor.pkl")
pipe.named_steps["xgb"].save_model("xgb_seq_meta.model")
print("Saved preprocessor and model.")

Saved preprocessor and model.


c:\Users\Marc\anaconda3\envs\casenv\lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [00:02:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
